##Pips

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


In [ ]:
from transformers import BertTokenizer, AutoTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import random
#note: importing something 2nd time does not cause any performance loss 


In [ ]:
from google.colab import files

In [ ]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

In [ ]:
logger = logging.getLogger('my-logger')
logger.propagate = False
# now if you use logger it will not log to console.

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

##Load Datasets

In [ ]:
path1 = '/content/drive/MyDrive/datasets/aug_train_augx2_cbuet_para.csv'
path2 = '/content/drive/MyDrive/datasets/aug_test.csv'
# path3 = '/content/drive/MyDrive/datasets/test.csv'

In [ ]:
aug_train = pd.read_csv(path1)
aug_test = pd.read_csv(path2)

In [ ]:
print(aug_train.shape, aug_test.shape)

(7014, 3) (1300, 3)


In [ ]:
aug_train.head(3)

,articleID,text,label
0,235,সমকামীতা'র হার বেড়েছে চট্টগ্রামে। নানা কারণে ব...,0
1,58271,৭ রোহিঙ্গা ধরে পুলিশে দিলো জনতা। সাতক্ষীরা সদর...,1
2,593,"['খালেদা জিয়ার শরীরে কামড় লেগে ঘা হয়েছে, তা...",0


In [ ]:
final_df = aug_train[['text','label']]
test_df = aug_test[['text','label']]

In [ ]:
final_df.shape, test_df.shape

((7014, 2), (1300, 2))

In [ ]:
#this is necessary because model won't run if there are NAN values. Though Training set is clean, this is for double check
final_df = final_df.dropna() 
final_df.label.value_counts()

0    3507
1    3507
Name: label, dtype: int64

In [ ]:
final_df.head()

,text,label
0,সমকামীতা'র হার বেড়েছে চট্টগ্রামে। নানা কারণে ব...,0
1,৭ রোহিঙ্গা ধরে পুলিশে দিলো জনতা। সাতক্ষীরা সদর...,1
2,"['খালেদা জিয়ার শরীরে কামড় লেগে ঘা হয়েছে, তা...",0
3,নিউটনের গতিবিদ্যা হল যৌন নিপীড়নকারীদের আশ্রয...,0
4,['গালিগালাজ স্বাস্থ্যের জন্য ভালো! । স্কুল - ক...,0


#prerequisites for model train/test

In [ ]:
# from transformers import BertTokenizer, AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# import time
# import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


#load the model you want to fine tune

In [ ]:
# from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

#All Test original model before fine-tune

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

##Testing 1: aug_test


In [ ]:
#test_df is loaded from load dataset section

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    650
1    650
Name: label, dtype: int64

In [ ]:
test_df.tail()

,text,label
1295,শতাব্দীর ‘পাগলামি’। বিনোদন ডেস্ক : ফুটপাতের ‍ও...,1
1296,"দিনাজপুরে ছাত্রদের ওপর পুলিশের গুলি, নিহত ১। স...",1
1297,খুলনা উপকূলে সতকর্তা জারি। ফাইল ছবি ঘূর্ণিঝড় ত...,1
1298,অবশেষে আইনজীবী পেলেন বাবু সোনার স্ত্রী ও প্রেম...,1
1299,কোটার দাবিতে প্রতিবন্ধী শিক্ষার্থীরাও শাহবাগে।...,1


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.48
  Test Loss: 0.69
  Test took: 0:00:41


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  621


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.43      0.13      0.20       650
           1       0.49      0.82      0.61       650

    accuracy                           0.48      1300
   macro avg       0.46      0.48      0.41      1300
weighted avg       0.46      0.48      0.41      1300



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

-0.06145350602517989


In [ ]:
print(roc_auc_score(y_true, y_score))

0.46115029585798817


##Testing 2: original test dataset of 1200


In [ ]:
path3 = '/content/drive/MyDrive/datasets/test.csv'
# path3 = 'translated_test.csv'
test_df = pd.read_csv(path3)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    600
1    600
Name: label, dtype: int64

In [ ]:
test_df.tail()

,label,text
1195,1,মোস্তফা কামালের 'থ্রি নভেলস' এখন ই-বুক ফরমেটেদ...
1196,1,‘জনসভা নিয়ে সরকারি দলের বিভিন্ন কথা অত্যন্ত দু...
1197,1,শয়নকক্ষ থেকে চুরি হওয়া নবজাতকের লাশ মিলল পুকুর...
1198,1,তারেক রহমানকে ফাঁসানোর ষড়যন্ত্র চলছে : মির্জা ...
1199,1,বিবিএস কেবলস লিমিটেড'র ডিলার কনফারেন্সকেবলস ম্...


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.53
  Test Loss: 0.69
  Test took: 0:00:37


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  630


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.20      0.29       600
           1       0.52      0.85      0.64       600

    accuracy                           0.53      1200
   macro avg       0.54      0.53      0.47      1200
weighted avg       0.54      0.53      0.47      1200



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

0.06629708788481584


In [ ]:
print(roc_auc_score(y_true, y_score))

0.41263055555555556


##Testing 3: translated_test


In [ ]:
path3 = '/content/drive/MyDrive/datasets/translated_test.csv'
# path3 = 'translated_test.csv'
test_df = pd.read_csv(path3)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    2000
1    2000
Name: label, dtype: int64

In [ ]:
test_df.tail()

,text,label
3995,ব্রাহ্মণবাড়িয়ায় ওএমএসের ৩০০বস্তা চালসহ ট্রাক আ...,1
3996,যুগ্ম-সচিব হলেন ১৫৪ কর্মকর্তা। উপসচিব পদমর্যাদ...,1
3997,স্প্রিকল ছিল বিশ্বাসঘাতক ও ঘৃণ্য ব্যক্তি : পুত...,1
3998,সাগরে ৩ দিন ভেসে থাকার পর ফিরে এলেন টোমি! ২৫ স...,1
3999,চীনা মুদ্রার দর আরো ১০% পড়তে পারে। চীনের আমদান...,1


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.51
  Test Loss: 0.69
  Test took: 0:02:08


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  2024


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.20      0.29      2000
           1       0.50      0.81      0.62      2000

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.45      4000
weighted avg       0.51      0.51      0.45      4000



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

0.0151733985222898


In [ ]:
print(roc_auc_score(y_true, y_score))

0.414252375


##Testing 4: custom fake


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/datasets/test_customfake.csv')
# test_df = pd.read_csv('test_customfake.csv')

In [ ]:
test_df.shape

(204, 2)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    102
1    102
Name: label, dtype: int64

In [ ]:
test_df.tail()

,text,label
199,বাবুগঞ্জে বৃদ্ধাকে পিটিয়ে হত্যার ঘটনায় গ্রেফতা...,1
200,দূষণ কমাতেই লোডশেডিং। বেশ কিছুদিন ধরেই দেশে বে...,0
201,পর্দায় চুমু খেতে খেতে ক্লান্ত নায়ক ইমরান হাসমি...,0
202,ঢাবির ‘চ’ ইউনিটে পাসের হার ১৯.৪৫ শতাংশ। নিজস্ব...,1
203,"দৃষ্টিহীন তরুণীকে ধর্ষণ, পুড়িয়ে ফেলা হলো ভ্রূণ...",1


In [ ]:
# test_df = test_df[test_df['label']==0]
# test_df.label.value_counts()

In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

We can see some of the results by the model here. Our model trains on half of the dataset and achieves around 0.80 in overall f1. Its likely that the model is trained longer it will achieve better performance. I might retrain it later on full data.

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.48
  Test Loss: 0.70
  Test took: 0:00:06


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  98


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.43      0.12      0.18       102
           1       0.49      0.84      0.62       102

    accuracy                           0.48       204
   macro avg       0.46      0.48      0.40       204
weighted avg       0.46      0.48      0.40       204



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

-0.05698028822981897


In [ ]:
print(roc_auc_score(y_true, y_score))

0.4627066512879662


#MODEL TRAINING 1 (augx2+fake|real data)

In [ ]:
max_len = 0
max_len_itr =0

for i in range(len(final_df)):
  sent = sent = final_df['text'][i]

  # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
  input_ids = tokenizer.encode(sent, add_special_tokens=True)

  # Update the maximum sentence length.
  leng = len(input_ids)
  if(leng>max_len):
    max_len = leng
    max_len_itr = i

print('Max sentence length: ', max_len)
print('sentence: ', final_df['text'][max_len_itr])

Token indices sequence length is longer than the specified maximum sequence length for this model (597 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  8218
sentence:  ["বদিবৃত্তান্ত : এক দশকে আবদুর রহমান বদির নানা ঘটনাবলি । বেশ কয়েক বছর ধরেই বাংলাদেশে সবচেয়ে ভয়াল মাদকের তালিকা ইয়াবা তার'বাবা'নামে এটি ছড়িয়ে পড়েছে দেশের বিভিন্ন স্তরে বিশেষ করে তরুণদের মাঝে । সারা দেশে ইয়াবার মূল যোগান আসে মুলত মায়ানমারের সীমান্তবর্তী বাংলাদেশের সর্বদক্ষিণের এলাকা টেকনাফ থেকে । আর ইয়াবার সাথে জড়িয়ে সবচেয়ে বেশি যে নামটা শোনা যায়, তিনি কক্সবাজার - ৪ আসন থেকে ২০০৮ এবং ২০১৪ দুই মেয়াদে সাংসদ হিসেবে দায়িত্ব পালন করা সাংসদ আবদুর রহমান ইউসুফ । নিঃসন্দেহে টেকনাফ - উখিয়া এলাকার সবচেয়ে আলোচিত ও সমালোচিত মানুষ তিনি । একাধিকবার স্বরাষ্ট্র মন্ত্রণালয়সহ সরকারের একাধিক মাদক ব্যবসায়ীর তালিকার শীর্ষে এসেছে আবদুর রহমান বদির নাম । বদি এবং ইয়াবার এই যুগলবন্দী বুঝতে ক্যালেন্ডারের পাতা পিছনের দিকে উল্টাতে হবে প্রায় পঞ্চাশ বছর । যেতে হবে সত্তরের দশকে বদির বাবা এজাহার মিয়া কোম্পানির সময়ে । ১৬ ফেব্রুয়ারি, ২০১৭ তারিখে বাংলাদেশ প্রতিদিনে প্রকাশিত একটি খবরে জানা যায়, অক্ষরজ্ঞান না থাকলেও দীর্ঘদিন তিনি ছিলেন টেকনাফ অঞ্চলের রাজনীতির নিয

In [ ]:
label_list = []
for label in final_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in final_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', final_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
# from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.85 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

5,961 training samples
1,053 validation samples


In [ ]:
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:

##this section HAS BEEN MOVED UP


# # from transformers import BertForSequenceClassification, AdamW, BertConfig

# # Load BertForSequenceClassification, the pretrained BERT model with a single 
# # linear classification layer on top. 
# model = BertForSequenceClassification.from_pretrained(
#     "Tahsin-Mayeesha/bangla-fake-news-mbert", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 2, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
# )

# # Tell pytorch to run this model on the GPU.
# model.cuda()

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:
# from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    373.    Elapsed: 0:01:00.
  Batch    80  of    373.    Elapsed: 0:01:59.
  Batch   120  of    373.    Elapsed: 0:02:59.
  Batch   160  of    373.    Elapsed: 0:03:59.
  Batch   200  of    373.    Elapsed: 0:04:58.
  Batch   240  of    373.    Elapsed: 0:05:57.
  Batch   280  of    373.    Elapsed: 0:06:57.
  Batch   320  of    373.    Elapsed: 0:07:56.
  Batch   360  of    373.    Elapsed: 0:08:56.

  Average training loss: 0.23
  Training epcoh took: 0:09:15

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.10
  Validation took: 0:00:34

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    373.    Elapsed: 0:01:00.
  Batch    80  of    373.    Elapsed: 0:01:59.
  Batch   120  of    373.    Elapsed: 0:02:59.
  Batch   160  of    373.    Elapsed: 0:03:58.
  Batch   200  of    373.    Elapsed: 0:04:58.
  Batch   240  of    373.    Elapsed: 0:05:57.
  Batch   280  of    373.    Elapsed: 0:06:57.
  Batch  

In [ ]:
#TM_fnfn : tahsin mayeesha finetune+finetune (first fine tuned with regular train, then model summarized train data)
model.save_pretrained('/content/drive/MyDrive/saved_models/mbert_aug_fk_augx2_cbuet_para/') #aug-fk means trained with augmented data + fake data

#load model from saved


In [ ]:
#don't run this cell if you aren't starting from here
#loading models
loaded_model = BertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/saved_models/mbert_aug_fk_augx2_cbuet_para/",
    num_labels = 2
)

# Tell pytorch to run this model on the GPU.
model = loaded_model
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

#All Test your fine tuned model

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

##Testing 1: aug_test


In [ ]:
aug_test = pd.read_csv('/content/drive/MyDrive/datasets/aug_test.csv')
test_df = aug_test[['text','label']]

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    650
1    650
Name: label, dtype: int64

In [ ]:
test_df.tail()

,text,label
1295,শতাব্দীর ‘পাগলামি’। বিনোদন ডেস্ক : ফুটপাতের ‍ও...,1
1296,"দিনাজপুরে ছাত্রদের ওপর পুলিশের গুলি, নিহত ১। স...",1
1297,খুলনা উপকূলে সতকর্তা জারি। ফাইল ছবি ঘূর্ণিঝড় ত...,1
1298,অবশেষে আইনজীবী পেলেন বাবু সোনার স্ত্রী ও প্রেম...,1
1299,কোটার দাবিতে প্রতিবন্ধী শিক্ষার্থীরাও শাহবাগে।...,1


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.99
  Test Loss: 0.05
  Test took: 0:00:42


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  1283


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       650
           1       1.00      0.98      0.99       650

    accuracy                           0.99      1300
   macro avg       0.99      0.99      0.99      1300
weighted avg       0.99      0.99      0.99      1300



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

0.9739856339655163


In [ ]:
print(roc_auc_score(y_true, y_score))

0.5648189349112427


##Testing 2: original test dataset of 1200


In [ ]:
path3 = '/content/drive/MyDrive/datasets/test.csv'
# path3 = 'translated_test.csv'
test_df = pd.read_csv(path3)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    600
1    600
Name: label, dtype: int64

In [ ]:
test_df.tail()

,label,text
1195,1,মোস্তফা কামালের 'থ্রি নভেলস' এখন ই-বুক ফরমেটেদ...
1196,1,‘জনসভা নিয়ে সরকারি দলের বিভিন্ন কথা অত্যন্ত দু...
1197,1,শয়নকক্ষ থেকে চুরি হওয়া নবজাতকের লাশ মিলল পুকুর...
1198,1,তারেক রহমানকে ফাঁসানোর ষড়যন্ত্র চলছে : মির্জা ...
1199,1,বিবিএস কেবলস লিমিটেড'র ডিলার কনফারেন্সকেবলস ম্...


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.92
  Test Loss: 0.43
  Test took: 0:00:38


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  1107


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.99      0.93       600
           1       0.98      0.86      0.92       600

    accuracy                           0.92      1200
   macro avg       0.93      0.92      0.92      1200
weighted avg       0.93      0.92      0.92      1200



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

0.852045487155812


In [ ]:
print(roc_auc_score(y_true, y_score))

0.5294277777777778


##Testing 3: translated_test


In [ ]:
path3 = '/content/drive/MyDrive/datasets/translated_test.csv'
# path3 = 'translated_test.csv'
test_df = pd.read_csv(path3)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    2000
1    2000
Name: label, dtype: int64

In [ ]:
test_df.tail()

,text,label
3995,ব্রাহ্মণবাড়িয়ায় ওএমএসের ৩০০বস্তা চালসহ ট্রাক আ...,1
3996,যুগ্ম-সচিব হলেন ১৫৪ কর্মকর্তা। উপসচিব পদমর্যাদ...,1
3997,স্প্রিকল ছিল বিশ্বাসঘাতক ও ঘৃণ্য ব্যক্তি : পুত...,1
3998,সাগরে ৩ দিন ভেসে থাকার পর ফিরে এলেন টোমি! ২৫ স...,1
3999,চীনা মুদ্রার দর আরো ১০% পড়তে পারে। চীনের আমদান...,1


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.92
  Test Loss: 0.36
  Test took: 0:02:08


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  3698


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.88      0.92      2000
           1       0.89      0.97      0.93      2000

    accuracy                           0.92      4000
   macro avg       0.93      0.92      0.92      4000
weighted avg       0.93      0.92      0.92      4000



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

0.8525372688159017


In [ ]:
print(roc_auc_score(y_true, y_score))

0.47590024999999997


##Testing 4: custom fake


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/datasets/test_customfake.csv')
# test_df = pd.read_csv('test_customfake.csv')

In [ ]:
test_df.shape

(204, 2)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    102
1    102
Name: label, dtype: int64

In [ ]:
test_df.tail()

,text,label
199,বাবুগঞ্জে বৃদ্ধাকে পিটিয়ে হত্যার ঘটনায় গ্রেফতা...,1
200,দূষণ কমাতেই লোডশেডিং। বেশ কিছুদিন ধরেই দেশে বে...,0
201,পর্দায় চুমু খেতে খেতে ক্লান্ত নায়ক ইমরান হাসমি...,0
202,ঢাবির ‘চ’ ইউনিটে পাসের হার ১৯.৪৫ শতাংশ। নিজস্ব...,1
203,"দৃষ্টিহীন তরুণীকে ধর্ষণ, পুড়িয়ে ফেলা হলো ভ্রূণ...",1


In [ ]:
# test_df = test_df[test_df['label']==0]
# test_df.label.value_counts()

In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
# print('Original: ', test_df['text'][0])
# print('Token IDs:', input_ids[0])

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = 16 # Evaluate with this batch size.
        )

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
#changedhere
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []
y_score = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        #changedhere
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        y_sc, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_sc = torch.nn.functional.softmax(y_sc)
        y_sc = y_sc.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        y_score.extend(y_sc)
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.86
  Test Loss: 0.80
  Test took: 0:00:07


In [ ]:
#how many correct value did the model predict?
cnt = 0
for x in range(len(y_pred)):
  if(y_true[x]==y_pred[x]):
    cnt = cnt+1

print('correct prediction: ',cnt)


correct prediction:  175


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.73      0.84       102
           1       0.78      0.99      0.87       102

    accuracy                           0.86       204
   macro avg       0.88      0.86      0.86       204
weighted avg       0.88      0.86      0.86       204



In [ ]:
print(matthews_corrcoef(y_true, y_pred))

0.7421597089520094


In [ ]:
print(roc_auc_score(y_true, y_score))

0.8261245674740484
